In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import seaborn as sns

from blazeface.dataset import input_dataset, anchors, target_encoder, prediction_decoder, utils
from blazeface.model import losses
from blazeface.constants import N_LANDMARKS

# Load data

In [ ]:
# take small subset of the training set to analyze
data_train, info = input_dataset.load_the300w_lp(split="train[:10%]")

In [ ]:
for x in data_train.take(1):
    break

print(x.keys())

# Visualize raw labels

In [ ]:
def visualize_landmarks(sample, ax, landmarks=None):
    """
    Args:
        sample (dict): must contain keys 'image' and 'landmarks_2d'.
        ax (AxesSubplot):
        landmarks (np.ndarray): if given, it will override 'landmarks_2d' in sample.
    """
    img = sample['image']
    if landmarks is None:
        landmarks = sample['landmarks_2d'].numpy()
    shape = tf.shape(img).numpy()
    ax.scatter(landmarks[:,0] * shape[0], landmarks[:,1] * shape[1], alpha=0.6, s=2, c='red');

In [ ]:
n_rows = 5
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axes = np.ravel(axes)

for i, x in enumerate(data_train.take(n_rows * n_cols)):
    ax = axes[i]
    ax.imshow(x['image'])
    visualize_landmarks(x, ax)

# Visualize preprocessed data (inputs to BlazeFace)

In [ ]:
def visualize_bbox(sample, ax):
    img = sample['image']
    shape = tf.shape(img).numpy()
    x1, y1, x2, y2 = input_dataset.landmarks_to_bboxes(x['landmarks_2d']).numpy()
    x1 *= shape[1]
    y1 *= shape[0]
    x2 *= shape[1]
    y2 *= shape[0]
    rect = Rectangle((x1, y1), x2 - x1, y2 - y1, fc="None", ec='green')
    ax.add_patch(rect)

In [ ]:
n_rows = 5
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axes = np.ravel(axes)

for i, x in enumerate(data_train.take(n_rows * n_cols)):
    ax = axes[i]
    ax.imshow(x['image'])
    # visualize bbox
    visualize_bbox(x, ax)
    # visualize landmarks
    landmarks_2d = input_dataset.reduce_landmarks(x['landmarks_2d']).numpy()
    visualize_landmarks(x, ax, landmarks=landmarks_2d)

# Visualize anchors

In [ ]:
all_anchors = anchors.generate_anchors()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

for i, loc in enumerate(all_anchors):
    x1, y1, w, h = loc
    c = mcolors.CSS4_COLORS[list(mcolors.CSS4_COLORS.keys())[int(i % len(mcolors.CSS4_COLORS.keys()))]]
    rect = Rectangle((x1 - w/2, y1-h/2), w, h, fc="None", ec=c, alpha=0.9, lw=0.5)
    ax.add_patch(rect)

# Create input dataset

In [ ]:
ds = data_train
ds = ds.map(input_dataset.unpack_dct)
ds = ds.map(input_dataset.preprocess_image_and_pass_landmarks)
ds = ds.map(lambda img, lmarks: (img, input_dataset.landmarks_to_bboxes(lmarks), input_dataset.reduce_landmarks(lmarks)))

ds = ds.batch(12)

# Visualize bounding boxes and landmark coordinate distributions

In [ ]:
bboxes = []
lmarks = []
for sample_batch in ds.take(1000):
    bboxes += sample_batch[1].numpy().tolist()
    lmarks += sample_batch[2].numpy().tolist()

bboxes = np.array(bboxes)
lmarks = np.array(lmarks)

bboxes = np.squeeze(bboxes)
lmarks = np.squeeze(lmarks)

bboxes.shape, lmarks.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax.set_title("BBox Coordinate Distributions", fontsize=16)
sns.kdeplot(x=bboxes[:,0], y=bboxes[:,1], cmap='coolwarm', fill=True, thresh=0.01, ax=ax)
sns.kdeplot(x=bboxes[:,2], y=bboxes[:,3], cmap='coolwarm', fill=True, thresh=0.01, ax=ax)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('bbox x', fontsize=13)
ax.set_ylabel('bbox y', fontsize=13)
ax.invert_yaxis();

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax.set_title("Landmark Coordinate Distributions", fontsize=16)

sns.kdeplot(x=np.reshape(lmarks, (-1, 2))[:, 0], y=np.reshape(lmarks, (-1, 2))[:, 1], cmap='coolwarm', fill=True, thresh=0.01, ax=ax)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('landmark x', fontsize=13)
ax.set_ylabel('landmark y', fontsize=13)
ax.invert_yaxis();

# Visualize positive anchors

In [ ]:
ds = ds.map(lambda img, bboxes, lmarks: (img, target_encoder.calculate_targets(all_anchors, bboxes, lmarks)))

In [ ]:
for sample_batch in ds.take(1):
    break

deltas = sample_batch[1]['deltas']
labels = sample_batch[1]['labels']

sample_batch[0].shape, [e.shape for e in sample_batch[1].values()]

In [ ]:
n_rows = 3
n_cols = 4

fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
colors = dict(zip(range(len(mcolors.CSS4_COLORS)), mcolors.CSS4_COLORS.values()))

def scale_coordinates(img, x1, y1, x2, y2):
    x1 *= img.shape[1]
    y1 *= img.shape[0]
    x2 *= img.shape[1]
    y2 *= img.shape[0]
    return x1, y1, x2, y2

for i, ax in enumerate(np.ravel(axes)):
    img = sample_batch[0][i]
    ax.imshow(img)

    for ci, pos_anchor in enumerate(all_anchors[tf.cast(labels[i, :, 0], dtype=tf.bool)]):
        x1, y1, x2, y2 = utils.xywh_to_xyxy(pos_anchor)
        x1, y1, x2, y2 = scale_coordinates(img, x1, y1, x2, y2)
        rect = Rectangle((x1, y1), x2 - x1, y2 - y1, fc="None", ec=colors[5*ci])
        ax.add_patch(rect)

# Apply decoder and test loss calculation

In [ ]:
dec = prediction_decoder.get_bboxes_and_landmarks_from_deltas(all_anchors, deltas)

dec[0][np.reshape(labels, (12, 896))[0] > 0]

In [ ]:
reg_loss = losses.RegressionLoss()
reg_loss(deltas, deltas + tf.random.normal(deltas.shape, 0, 0.5, dtype=tf.float32))

In [ ]:
class_loss = losses.ClassLoss()
class_loss(labels, tf.cast(tf.random.uniform(labels.shape, 0, 1, dtype=tf.float32), dtype=tf.float32))